In [1]:
import numpy as np
from collections import Counter

class DecisionTreeNode:
    def __init__(self, feat_num=None, threshold=None, left=None, right=None, pred_class=None, prob_class1=None):
        self.left, self.right = left, right
        self.pred_class, self.prob_class1 = pred_class, prob_class1
        self.feat_num = feat_num
        self.threshold = threshold

def find_best_split(X, y):
    best_split = None
    best_gain = -1

    for j in range(X.shape[1]):
        unique_val = np.unique(X[:, j])
        for c in unique_val:
            l_index = X[:, j] >= c
            r_index = X[:, j] < c
            splits = [{'subset': y[l_index]}, {'subset': y[r_index]}]

            gain_ratio = igratio(y, splits)
            if gain_ratio > best_gain:
                best_gain = gain_ratio
                best_split = {'feat_num': j, 'threshold': c, 'splits': splits}

    return best_split

def entropy(y):
    if (len(y) == 0):
        return 0
    counts = np.bincount(y)
    prob = counts / len(y)
    return -np.sum([p * np.log2(p) for p in prob if p > 0])

def igratio(y, splits):
    total_entropy = entropy(y)
    weighted_entropy = split_info = 0

    for i in splits:
        subset = i['subset']
        split_weight = len(subset) / len(y)
        weighted_entropy += split_weight * entropy(subset)
        if split_weight > 0:
          split_info -= split_weight * np.log2(split_weight)
        else:
          split_info -=0

    gain = total_entropy - weighted_entropy
    if split_info == 0:
        return 0
    return gain / split_info

def build_dtree(X, y):
    if len(np.unique(y)) == 1:
        return DecisionTreeNode(pred_class=y[0], prob_class1=1 if y[0] == 1 else 0)

    if len(y) == 0:
        return DecisionTreeNode(pred_class=1)
    best_split = find_best_split(X, y)

    if best_split is None or best_split['splits'][0]['subset'].size == 0 or best_split['splits'][1]['subset'].size == 0:
        return DecisionTreeNode(pred_class=1 if np.sum(y) >= len(y) / 2 else 0)

    left_node = build_dtree(X[best_split['splits'][0]['subset']], best_split['splits'][0]['subset'])
    right_node = build_dtree(X[best_split['splits'][1]['subset']], best_split['splits'][1]['subset'])

    return DecisionTreeNode(feat_num=best_split['feat_num'], threshold=best_split['threshold'], left=left_node, right=right_node)

def predict_tree(node, x):
    if node.pred_class is not None:
        return node.pred_class

    if x[node.feat_num] >= node.threshold:
        return predict_tree(node.left, x)
    else:
        return predict_tree(node.right, x)

#Testing the code:

x = np.array([[1, 1], [0, 1], [2, 6], [0, 6]])
y = np.array([0, 1, 1, 0])
x_test = np.array([[1, 3], [0, 0], [0, 6]])

root_node = build_dtree(x, y)
print("y_test:", [predict_tree(root_node, i) for i in x_test])

y_test: [0, 1, 1]
